In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
!pip install crewai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install -U duckduckgo-search

  Attempting uninstall: duckduckgo-search
    Found existing installation: duckduckgo_search 6.3.3
    Uninstalling duckduckgo_search-6.3.3:
      Successfully uninstalled duckduckgo_search-6.3.3

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchRun
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [11]:
import os

os.environ["OPENAI_API_KEY"] = "secNbxX4Pg4wp8rscewBm1kSis1tFimU"
os.environ["OPENAI_API_BASE"] = "https://Phi-3-5-mini-instruct-iplvi.eastus2.models.ai.azure.com/chat/completions"
os.environ["OPENAI_MODEL_NAME"] = "Phi-3-5-mini-instruct-iplvi"

In [30]:
import requests

# Define the API endpoint and headers
api_endpoint = "https://Phi-3-5-mini-instruct-iplvi.eastus2.models.ai.azure.com/chat/completions"
api_key = "secNbxX4Pg4wp8rscewBm1kSis1tFimU"  # Replace with your Azure API key

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
    "api-version": "2023-05-15"  # Update this if a newer version is available
}

# Define the request payload
payload = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! How can I use this API?"}
    ],
    "max_tokens": 100,
    "temperature": 0.7,
    "top_p": 0.95
}

# Make the POST request
response = requests.post(api_endpoint, json=payload, headers=headers)

# Handle the response
if response.status_code == 200:
    result = response.json()
    print("Response:", result)
else:
    print(f"Error: {response.status_code}, {response.text}")


Response: {'choices': [{'finish_reason': 'length', 'index': 0, 'message': {'content': " Hello! To use an API (Application Programming Interface), you typically need to follow these steps:\n\n1. **Read the API Documentation:**\n   - Find the official documentation for the API. This will provide you with all the necessary information about how to interact with the API, including endpoints, request methods, parameters, authentication, response formats, and more.\n\n2. **Understand the API's Endpoints:**\n   - Identify which endpoints", 'role': 'assistant', 'tool_calls': []}}], 'created': 1731699455, 'id': 'chat-f5bd8e27884242c7b6356d82f709949a', 'model': 'phi35-mini-instruct', 'object': 'chat.completion', 'usage': {'completion_tokens': 100, 'prompt_tokens': 20, 'total_tokens': 120}}


In [16]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [17]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [18]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [19]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [26]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [27]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [28]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [29]:
result = crew.kickoff(inputs={"topic": "Central Bank Digital Currency"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Central Bank Digital Currency.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


RuntimeError: no validator found for <class 'langchain_core.runnables.base.Runnable'>, see `arbitrary_types_allowed` in Config